# Importing Libreries

In [33]:
# Suppress warnings in Jupyter Notebooks
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
plt.style.use('ggplot')

import numpy as np
import pandas as pd
import h2o

import time
from h2o.automl import H2OAutoML

import scikitplot as skplt #pip install scikit-plot
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import cohen_kappa_score, confusion_matrix

# Initializing H2O Cluster

In [ ]:
h2o.init()

# Readinng data

In [44]:
df=pd.read_csv("train_data.csv" )

In [ ]:
print(); print(df.shape)
print(); print(df.head())
print(); print(df.columns.values)

In [46]:
del df["Unnamed: 0"]

In [ ]:
df

In [ ]:
df.groupby(["class"]).count()

In [ ]:
df.head()

### getting data in h2o frame

In [ ]:
hf = h2o.H2OFrame(df)

### Train validation division

In [ ]:
# Data Transform - Split train : test datasets
train, valid = hf.split_frame(ratios = [.80], seed = 1234)
print("Training Dataset", train.shape)
print("Validation Dataset", valid.shape)

In [ ]:
train.head(5)

In [ ]:
valid.head(5)

# storing target column

In [54]:
# Identify predictors and response 
featureColumns = train.columns
targetColumn   = "class"
featureColumns.remove(targetColumn)

In [ ]:
print("Feature Columns : ");     print(featureColumns)
print("\n\nTarget Column : ");   print(targetColumn)

# Training the model

In [ ]:
localtime = time.asctime( time.localtime(time.time()) )
print("Local current time :", localtime)
print()
        
# Run AutoML for YY base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=15, seed=1234, exclude_algos = ["StackedEnsemble"],
                balance_classes = True
               )
aml.train(x=featureColumns, y=targetColumn, training_frame = train, validation_frame = valid)

localtime = time.asctime( time.localtime(time.time()) )
print()
print("Local current time :", localtime)

# choosing the best model

In [ ]:
lb = aml.leaderboard
print(lb.head(rows = lb.nrows))

# Using the best model

In [58]:
# Evaluate the best model with testing data.
model = aml.leader 

# Accuracy on validation set

In [ ]:
predicted_y = model.predict(valid[featureColumns])

predicted_data = predicted_y.as_data_frame()
valid_dataset = valid.as_data_frame()

# Evaluate the skill of the Trained model
acc                 = accuracy_score(valid_dataset[targetColumn], predicted_data['predict'])
classReport         = classification_report(valid_dataset[targetColumn], predicted_data['predict'])
confMatrix          = confusion_matrix(valid_dataset[targetColumn], predicted_data['predict']) 
        
print(); print('Testing Results of the trained model: ')
print(); print('Accuracy : ', acc)
print(); print('Confusion Matrix :\n', confMatrix)
print(); print('Classification Report :\n',classReport)

# Confusion matrix
skplt.metrics.plot_confusion_matrix(valid_dataset[targetColumn], predicted_data['predict'], figsize=(7,7)); plt.show()

In [ ]:
print(); print();
#pd.set_option("display.max_rows", None, "display.max_columns", None)
VI = model.varimp(use_pandas=True)
print(VI[['variable', 'percentage']].head(25))

print()
#pd.set_option("display.max_rows", None, "display.max_columns", None)
VI = model.varimp(use_pandas=True)
VI = VI[['variable', 'percentage']]
VI.head(25).plot(x = 'variable', y = 'percentage', kind = 'barh', figsize = (14,16), grid = True,
                 title = "Variable Importance: H2O model")
plt.gca().invert_yaxis(); plt.show()

# Saving the model

In [ ]:
model_path = h2o.save_model(model=model, force=True)
print(model_path)

# Uploading the binary of the model to the H2O cluster

In [62]:
# download the model built above to your local machine
my_local_model = h2o.download_model(model, path="/home/sysadm/Documents/sem_2_Biswajit_rana/RKMVERI_2ND/class/ETBDC/h2O/UPLOAD")

# upload the model that you just downloded above
# to the H2O cluster
uploaded_model = h2o.upload_model(my_local_model)

# Uploading the test data to the H2O Cluster

In [ ]:
h2o.upload_file("test_data.csv", destination_frame="test_data_")